In [6]:
%%html
<link rel="stylesheet" type="text/css" href="theme/sixty_north.css">

# The Elm kernel for Jupyter
## Interactive notebooks for Elm

----

**Austin Bingham**<br>
*twitter:* @austin_bingham<br>
*email:* austin@sixty-north.com

----

![Sixty North logo](theme/sixty-north-logo.small.png)

# What is a Jupyter notebook?

> The Jupyter Notebook is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations and explanatory text.
>
> -- <cite>jupyter.org</cite>

Let's look at [an example showing the Elm architecture](../../examples/the-elm-architecture.ipynb).

# The Elm kernel
## Support for Elm code cells in Jupyter notebooks
----
- Accumulates code cells
- Compiles accumulated code on `--compile-code`
- Compilation goes into a temporary file
- Compilaton results are shipped back to the web client
- `elm-stuff` if kept between compilations

# Kernel initialization
----

```python
class ElmKernel(Kernel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._code = []
        self._tempdir = TemporaryDirectory()
```

# Compilation and execution

## Receive code from the web client
----
```python
def do_execute(self, code, . . .):
    self._code.append(code)
    if self._should_compile:
        try:
            code = "\n".join(self._code)
            self._code = []
            self._compile(code)
        except Exception as exc:
            self._send_error_result(str(exc))
            return {'status': 'error' . . . }
    return {'status': 'ok' . . . }
```

## Write code to file and compile it, reporting any failures
----
```python
def _compile(self, code):
    with self._tempfile('input.elm') as infile,\
         self._tempfile('index.js') as outfile:
        with open(infile, mode='wt') as f:
            f.write(code)
        try:
            # compile in a subprocess
        except subprocess.CalledProcessError as err:
            self._send_error_result(err.stdout)
        except Exception as err:
            self._send_error_result(repr(err))
            raise
```

## Run compiler, read output, and report success
----
```python
subprocess.run(
    ['elm-make', infile, '--yes', '--output={}'.format(outfile)],
    cwd=self._tempdir.name,
    check=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    encoding=sys.getdefaultencoding())

with open(outfile, mode='rt') as f:
    javascript = f.read()

self._send_success_result(javascript)
```

# Reporting success
## Producing JavaScript embedding code
----
```python
module_name = "Main"
div_id = 'elm-div-' + str(self.execution_count)
template = """
    var defineElm = function(cb) {{
        if (this.Elm) {{
            this.oldElm = this.Elm;
        }}
        var define = null;
        {js}
        cb();
    }};
    var obj = new Object();
    defineElm.bind(obj)(function(){{
        var mountNode = document.getElementById('{div_id}');
        obj.Elm. {module_name}.embed(mountNode);
    }});"""
```

## Injecting HTML into the client
----
```python
self.send_response(
    self.iopub_socket,
    'display_data',
    {
        'metadata': {},
        'data': {
            'text/html': '<div id="' + div_id + '"></div>'
        }
    }
)
```

## Sending JavaScript to client
----
```python
javascript = template.format(
    js=javascript,
    module_name=module_name,
    div_id=div_id)

self.send_response(
    self.iopub_socket,
    'display_data',
    {
        'metadata': {},
        'data': {
            'application/javascript': javascript
        }
    })
```

# Good news! It seems to work!

![Agent Cooper giving thumbs up](images/cooper-thumbsup.jpeg)